In [3]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [4]:
# Import Depencies

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import numpy as np
import requests
from pprint import pprint
import time
import json
from requests.structures import CaseInsensitiveDict

from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [5]:
#Section for user input

lat = ''
lon = ''

default = [0, 0]
cleveland = [41.4993, -81.6944]
atlanta = [33.7488, -84.3877]
los_angeles = [34.0549, -118.2426]
new_york_city = [40.7128, -74.0060]
chicago = [41.8781, -87.6298]
seattle = [47.6061, -122.3328]

location_search = [123456789,123456789] #user_defined or written to pre-determined

default_city_list =  {'Cleveland' : cleveland , 'Atlanta' : atlanta, 'Los Angeles' : los_angeles, 'New York City' : new_york_city, 'Chicago' : chicago, 'Seattle' : seattle}


print('Available list of cities\n')
for k, v in default_city_list.items():
    print(k)



Available list of cities

Cleveland
Atlanta
Los Angeles
New York City
Chicago
Seattle


In [7]:
user_coordinates = input("Would you like pick from a list of available cities? yes or no")

if user_coordinates.lower() == 'y' or user_coordinates.lower() == 'yes':

    match = input("Please pick a city from the list.\n")

    for k, v in default_city_list.items():
        if match.lower() == k.lower():
            location_search = v

elif user_coordinates.lower() == 'n' or user_coordinates.lower() == 'no':
    user_lat = input("Please enter the latitude. ")
    user_lon = input("Please enter the longitude. ")
    location_search = [float(user_lat), float(user_lon)]

else:
    raise Exception("Invalid Input. Must specify Yes (y) or No (n)")

if location_search == [123456789, 123456789]:
    raise Exception("Invalid location. Use a name that is in the list, or choose No (n) your own coordinates.")
    
print(f'Using coordinates {location_search}\n')


Using coordinates [41.4993, -81.6944]



In [12]:



#get latitude, longitude, and radius for starting point - New York City is [40.712776 N, -74.005974 W], Germany is [51.1657 N, 10.4515 E]  Germany country code is "de"
#London is [51.507351, -0.127758] Brisbane is [-27.469770, 153.025131]
#I can find London, but US cities are not coming up





#THESE VALUES ARE JUST FOR TESTING. DELETE 
#location_search = germany
#lat = "-27.469770"
#lon = "153.025131"

#User inputs should be placed for these too - Just ask if the default values are OK.
radius_meters = 5000
country_code = ''
limit = 100
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [13]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [14]:
# dictionary of each category that will be captured
# we also may want to include user input for this too. Loop through these, or other questions. Set value to True. Default value is false. Must have at least one of these active in order
# to run the script. If not, throw an error
# We either need to specify that the list needs to be under 500, because that is the max number, or create 
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# conditions, organic.only, vegetarian.only, wheelchair.yes
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"

# atms in radius, 

# contact information for the restaurants, opening/closing hours, web address, phone number

In [15]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json_info = resp.json()
pprint(json_info)

200
{'features': [{'geometry': {'coordinates': [-81.69677670688294,
                                            41.50747877025738],
                            'type': 'Point'},
               'properties': {'address_line1': 'Great Lakes Science Center',
                              'address_line2': '601 Erieside Avenue, '
                                               'Cleveland, OH 44114, United '
                                               'States of America',
                              'categories': ['building',
                                             'building.tourism',
                                             'entertainment',
                                             'entertainment.museum',
                                             'internet_access',
                                             'wheelchair',
                                             'wheelchair.yes'],
                              'city': 'Cleveland',
                              'countr

In [16]:
pd.json_normalize(json_info["features"]).columns.values

array(['type', 'properties.name', 'properties.country',
       'properties.country_code', 'properties.state', 'properties.county',
       'properties.city', 'properties.postcode', 'properties.district',
       'properties.street', 'properties.housenumber', 'properties.lon',
       'properties.lat', 'properties.state_code', 'properties.formatted',
       'properties.address_line1', 'properties.address_line2',
       'properties.categories', 'properties.details',
       'properties.datasource.sourcename',
       'properties.datasource.attribution',
       'properties.datasource.license', 'properties.datasource.url',
       'properties.datasource.raw.ele', 'properties.datasource.raw.name',
       'properties.datasource.raw.phone',
       'properties.datasource.raw.museum',
       'properties.datasource.raw.osm_id',
       'properties.datasource.raw.tourism',
       'properties.datasource.raw.website',
       'properties.datasource.raw.building',
       'properties.datasource.raw.osm_type'

In [17]:
features_pd = pd.json_normalize(json_info["features"])

In [18]:
properties_df = features_pd[["properties.address_line1", "properties.address_line2", "properties.place_id"]]
properties_df

,properties.address_line1,properties.address_line2,properties.place_id
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
...,...,...,...
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...
98,Kram & Kram Auto Repair,"East 55th Street, Cleveland, OH 44105, United ...",51fe44a40caf6954c05976911f70b3c04440f00102f901...


In [19]:
# Rename columns
properties_df = properties_df.rename(columns = {"properties.address_line1": "Property", "properties.address_line2": "Address2", "properties.place_id": "Place_ID"})
properties_df

,Property,Address2,Place_ID
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
...,...,...,...
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...
98,Kram & Kram Auto Repair,"East 55th Street, Cleveland, OH 44105, United ...",51fe44a40caf6954c05976911f70b3c04440f00102f901...


In [20]:
# Create separate columns for the address fields
properties_df[['Address', 'City', 'State_Zip', 'Country']] = properties_df.Address2.str.split(",", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
...,...,...,...,...,...,...,...
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America
98,Kram & Kram Auto Repair,"East 55th Street, Cleveland, OH 44105, United ...",51fe44a40caf6954c05976911f70b3c04440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America


In [21]:
properties_df = properties_df.dropna()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
...,...,...,...,...,...,...,...
94,M&I Deli,"East 55th Street, Cleveland, OH 44105, United ...",512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America


In [24]:
properties_df["State_Zip"] = properties_df['State_Zip'].str.strip()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
...,...,...,...,...,...,...,...
94,M&I Deli,"East 55th Street, Cleveland, OH 44105, United ...",512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America


In [25]:
properties_df.State_Zip.str.split(" ", expand = True)

,0,1
0,OH,44114
1,OH,44114
2,OH,44109
3,OH,44113
4,OH,44115
...,...,...
94,OH,44105
95,OH,44105
96,OH,44103
97,OH,44114


In [26]:
properties_df[['State', 'Zip']] = properties_df.State_Zip.str.split(" ", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America,OH,44114
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America,OH,44114
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America,OH,44109
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America,OH,44113
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America,OH,44115
...,...,...,...,...,...,...,...,...,...
94,M&I Deli,"East 55th Street, Cleveland, OH 44105, United ...",512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
95,Allen Hardware,"2733 East 55th Street, Cleveland, OH 44105, Un...",515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
96,Rite Aid,"East 53rd Street, Cleveland, OH 44103, United ...",51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America,OH,44103
97,A-To-Z Appliance and Furniture,"Commerce Avenue, Cleveland, OH 44114, United S...",51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America,OH,44114


In [27]:
properties_df = properties_df.drop('Address2', axis=1)
properties_df

,Property,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Great Lakes Science Center,51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America,OH,44114
1,International Women's Air and Space Museum,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America,OH,44114
2,A Christmas Story House Museum,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America,OH,44109
3,Frederich Bicycle,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America,OH,44113
4,Graffiti Caps,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America,OH,44115
...,...,...,...,...,...,...,...,...
94,M&I Deli,512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
95,Allen Hardware,515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,OH 44105,United States of America,OH,44105
96,Rite Aid,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,OH 44103,United States of America,OH,44103
97,A-To-Z Appliance and Furniture,51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,OH 44114,United States of America,OH,44114


In [28]:
properties_df = properties_df.drop('State_Zip', axis=1)
properties_df

,Property,Place_ID,Address,City,Country,State,Zip
0,Great Lakes Science Center,51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,United States of America,OH,44114
1,International Women's Air and Space Museum,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,United States of America,OH,44114
2,A Christmas Story House Museum,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,United States of America,OH,44109
3,Frederich Bicycle,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,United States of America,OH,44113
4,Graffiti Caps,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,United States of America,OH,44115
...,...,...,...,...,...,...,...
94,M&I Deli,512940a8d0bb6954c0592857231292bf4440f00102f901...,East 55th Street,Cleveland,United States of America,OH,44105
95,Allen Hardware,515706584bac6954c059fc9216f05fbf4440f00102f901...,2733 East 55th Street,Cleveland,United States of America,OH,44105
96,Rite Aid,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,East 53rd Street,Cleveland,United States of America,OH,44103
97,A-To-Z Appliance and Furniture,51aa379682bd6954c059bf63cda508c14440f00102f901...,Commerce Avenue,Cleveland,United States of America,OH,44114


In [29]:
properties_df = properties_df.reindex(['Place_ID', 'Property', 'Address', 'City', 'State', 'Zip', 'Country'], axis=1)
properties_df

,Place_ID,Property,Address,City,State,Zip,Country
0,51512b54fd976c54c059bad57810f5c04440f00102f901...,Great Lakes Science Center,601 Erieside Avenue,Cleveland,OH,44114,United States of America
1,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,International Women's Air and Space Museum,1501 North Marginal Road,Cleveland,OH,44114,United States of America
2,5164ffa097066c54c0594f6fd80affbb4440f00102f901...,A Christmas Story House Museum,1103 Rowley Avenue,Cleveland,OH,44109,United States of America
3,5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,Frederich Bicycle,3800 Lorain Avenue,Cleveland,OH,44113,United States of America
4,5151fb08579b6a54c059d240edeb0ac04440f00103f901...,Graffiti Caps,3200 Carnegie Avenue,Cleveland,OH,44115,United States of America
...,...,...,...,...,...,...,...
94,512940a8d0bb6954c0592857231292bf4440f00102f901...,M&I Deli,East 55th Street,Cleveland,OH,44105,United States of America
95,515706584bac6954c059fc9216f05fbf4440f00102f901...,Allen Hardware,2733 East 55th Street,Cleveland,OH,44105,United States of America
96,51b3fe3b47c96954c0592f97c66350c24440f00102f901...,Rite Aid,East 53rd Street,Cleveland,OH,44103,United States of America
97,51aa379682bd6954c059bf63cda508c14440f00102f901...,A-To-Z Appliance and Furniture,Commerce Avenue,Cleveland,OH,44114,United States of America


In [30]:
no_duplicates_df = properties_df.drop_duplicates(subset = "Place_ID", inplace = True)
no_duplicates_df

In [86]:
categories_df = features_pd[["properties.categories", "properties.place_id"]]
categories_df

,properties.categories,properties.place_id
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [88]:
# Rename columns
categories_df = categories_df.rename(columns = {"properties.categories": "Categories", "properties.place_id": "Place_ID"})
categories_df

,Categories,Place_ID
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [89]:
categories_df = categories_df.dropna()
categories_df

,Categories,Place_ID
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...
